In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/semeval2024task8/subtaskA_dev_monolingual.jsonl
/kaggle/input/semeval2024task8/subtaskA_train_monolingual.jsonl
/kaggle/input/semeval2024task8/subtaskA_train_multilingual.jsonl
/kaggle/input/semeval2024task8/subtaskB_train.jsonl
/kaggle/input/semeval2024task8/subtaskA_dev_multilingual.jsonl
/kaggle/input/semeval2024task8/subtaskC_dev.jsonl
/kaggle/input/semeval2024task8/subtaskB_dev.jsonl
/kaggle/input/semeval2024task8/subtaskC_train.jsonl
/kaggle/input/mini-updated-code/config.json
/kaggle/input/mini-updated-code/__results__.html
/kaggle/input/mini-updated-code/tokenizer_config.json
/kaggle/input/mini-updated-code/__notebook__.ipynb
/kaggle/input/mini-updated-code/model.safetensors
/kaggle/input/mini-updated-code/special_tokens_map.json
/kaggle/input/mini-updated-code/__output__.json
/kaggle/input/mini-updated-code/sentencepiece.bpe.model
/kaggle/input/mini-updated-code/custom.css
/kaggle/input/mini-updated-code/__results___files/__results___19_0.png


# task c


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, AdamW, set_seed
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
random_seed = 0
set_seed(random_seed)
train_path =  '/kaggle/input/semeval2024task8/subtaskC_train.jsonl'
test_path =  '/kaggle/input/semeval2024task8/subtaskC_dev.jsonl'

train_df = pd.read_json(train_path, lines=True)
eval_df = pd.read_json(test_path, lines=True)


2024-05-11 12:47:39.695898: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 12:47:39.695993: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 12:47:39.813912: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
train_df['new_label'] = train_df['label']

In [6]:
train_df

,id,text,label,new_label
0,5df7a120-f5ff-4881-8624-058f8a2fee5a,- Strengths:\r\n* Outperforms ALIGN in supervi...,88,88
1,063f9429-b94b-4e86-bd4c-c48e9bf85850,This paper addresses the problem of disambigua...,286,286
2,2759de87-3708-4f81-9e5f-ed132c292d2f,"- Strengths:\r\nGood ideas, simple neural lear...",62,62
3,25ef7554-db90-4212-b218-06679b25efb8,This paper presents several weakly supervised ...,35,35
4,8b51f390-6552-4982-b5d4-1b165e598df1,This paper describes a model for cross-lingual...,98,98
...,...,...,...,...
3644,35a52bbb-c5d8-4446-8e5c-b4521d895196,The paper presents a novel method for reducing...,4,4
3645,e8941b7e-e562-4c91-b051-678d0b4a885a,This paper addresses to reduce test-time compu...,10,10
3646,1faa5671-0ec6-4b9a-b539-c7381d09b518,I do need to see the results in a clear table....,37,37
3647,e0078728-7e68-467b-b843-0572211ad812,This paper explores a new quantization method ...,54,54


In [7]:
from transformers import AutoTokenizer
# Load tokenizer (replace with your chosen model)
tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [8]:
a = []
b = []
print(a)

[]


In [9]:
for i in range(len(train_df)):
    mapp = {x : tokenizer.encode(x, add_special_tokens=False) for x in train_df['text'][i].split()}
    sum_a = 0
    sum_b = 0
    label_index = train_df['new_label'][i]
    for j,x in enumerate(mapp):
        sum_a += len(mapp[x])
        if(j <= label_index):
            sum_b += len(mapp[x])
    a.append(sum_a)
    b.append(sum_b)

In [10]:
c = b
c = [x if x <= 512 else 512 for x in c]

In [11]:
train_df['label'] = c

In [12]:
train_df['label'].max()

512

In [13]:
train_df

,id,text,label,new_label
0,5df7a120-f5ff-4881-8624-058f8a2fee5a,- Strengths:\r\n* Outperforms ALIGN in supervi...,137,88
1,063f9429-b94b-4e86-bd4c-c48e9bf85850,This paper addresses the problem of disambigua...,455,286
2,2759de87-3708-4f81-9e5f-ed132c292d2f,"- Strengths:\r\nGood ideas, simple neural lear...",102,62
3,25ef7554-db90-4212-b218-06679b25efb8,This paper presents several weakly supervised ...,49,35
4,8b51f390-6552-4982-b5d4-1b165e598df1,This paper describes a model for cross-lingual...,170,98
...,...,...,...,...
3644,35a52bbb-c5d8-4446-8e5c-b4521d895196,The paper presents a novel method for reducing...,6,4
3645,e8941b7e-e562-4c91-b051-678d0b4a885a,This paper addresses to reduce test-time compu...,21,10
3646,1faa5671-0ec6-4b9a-b539-c7381d09b518,I do need to see the results in a clear table....,52,37
3647,e0078728-7e68-467b-b843-0572211ad812,This paper explores a new quantization method ...,78,54


In [14]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [15]:
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, XLMRobertaConfig, AdamW

tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
config = XLMRobertaConfig.from_pretrained("xlm-roberta-base")

# Define the path to your partially trained model
model_path = "/kaggle/input/mini-updated-code"

# Load the partially trained model
model = XLMRobertaForSequenceClassification.from_pretrained(model_path, config=config)
model = model.to(device)

In [16]:
model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [17]:
from transformers import XLMRobertaForSequenceClassification, XLMRobertaConfig
# Define the new model configuration
new_model_name = "xlm-roberta-base"
new_config = XLMRobertaConfig.from_pretrained(new_model_name)
new_config.num_labels = 512
new_model = XLMRobertaForSequenceClassification(config=new_config)

In [18]:
new_model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [19]:
# Copy the parameters from the pretrained model to the new model, except the classifier layer
pretrained_state_dict = model.state_dict()
new_model_state_dict = new_model.state_dict()

# Filter out the classifier layer parameters
classifier_params = {k: v for k, v in pretrained_state_dict.items() if 'classifier' in k}

# Update the new model's state dict with the parameters of the pretrained model, except the classifier layer
new_model_state_dict.update({k: v for k, v in pretrained_state_dict.items() if k not in classifier_params})


In [20]:
# Load the updated state dict into the new model
new_model.load_state_dict(new_model_state_dict)

<All keys matched successfully>

In [ ]:
# # Freeze all parameters except those of the classifier layer
# for name, param in new_model.named_parameters():
#     if 'classifier' not in name:  # If the parameter does not belong to the classifier layer
#         param.requires_grad = False

# # Verify that parameters are frozen as intended
# for name, param in new_model.named_parameters():
#     print(f'{name}: {param.requires_grad}')

In [21]:
new_model = new_model.to(device)

In [22]:
# Tokenize text data from DataFrame
train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True)
eval_encodings = tokenizer(eval_df['text'].tolist(), truncation=True, padding=True)

In [23]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to(device) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).to(device)
        return item

    def __len__(self):
        return len(self.labels)

In [24]:
class CustomDatasetTest(Dataset):
    def __init__(self, encodings, labels,text):
        self.encodings = encodings
        self.labels = labels
        self.text = text

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to(device) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).to(device)
        item['text'] = self.text[idx]
        return item
        

    def __len__(self):
        return len(self.labels)

In [25]:
# Define training parameters
batch_size = 8
learning_rate = 2.0e-5
num_epochs = 10

In [26]:
train_dataset = CustomDataset(train_encodings, train_df['label'].tolist())
eval_dataset = CustomDatasetTest(eval_encodings, eval_df['label'].tolist(),eval_df['text'].tolist())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=batch_size)

In [39]:
# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1.0e-3)
criterion1 = nn.CrossEntropyLoss()
criterion2 = nn.MSELoss()
criterion3 = nn.L1Loss()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [28]:
# for name, param in new_model.named_parameters():
#         param.requires_grad = True

In [29]:
# Verify that parameters are frozen as intended
for name, param in new_model.named_parameters():
    print(f'{name}: {param.requires_grad}')

roberta.embeddings.word_embeddings.weight: True
roberta.embeddings.position_embeddings.weight: True
roberta.embeddings.token_type_embeddings.weight: True
roberta.embeddings.LayerNorm.weight: True
roberta.embeddings.LayerNorm.bias: True
roberta.encoder.layer.0.attention.self.query.weight: True
roberta.encoder.layer.0.attention.self.query.bias: True
roberta.encoder.layer.0.attention.self.key.weight: True
roberta.encoder.layer.0.attention.self.key.bias: True
roberta.encoder.layer.0.attention.self.value.weight: True
roberta.encoder.layer.0.attention.self.value.bias: True
roberta.encoder.layer.0.attention.output.dense.weight: True
roberta.encoder.layer.0.attention.output.dense.bias: True
roberta.encoder.layer.0.attention.output.LayerNorm.weight: True
roberta.encoder.layer.0.attention.output.LayerNorm.bias: True
roberta.encoder.layer.0.intermediate.dense.weight: True
roberta.encoder.layer.0.intermediate.dense.bias: True
roberta.encoder.layer.0.output.dense.weight: True
roberta.encoder.layer.

In [30]:
train_loss = []
test_loss = []

In [40]:
# Function for training
def train(model, optimizer, criterion, dataloader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in tqdm(train_loader, leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            labels = labels.to(torch.float32)

            optimizer.zero_grad()

            outputs = new_model(input_ids, attention_mask=attention_mask)
            prediction = torch.argmax(outputs.logits, dim=1).to(torch.float32)
            prediction.requires_grad_(True)
            loss =torch.sqrt(criterion2(prediction,labels))

            loss.backward()
            optimizer.step()
            running_loss += loss.item() * input_ids.size(0)

            del input_ids, attention_mask, labels, outputs
            torch.cuda.empty_cache()
            
        epoch_loss = running_loss / len(dataloader.dataset)
        train_loss.append(epoch_loss)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

        


In [41]:
def find_position(text, encoded_pos,tokenizer):
    mapp = {x : tokenizer.encode(x, add_special_tokens=False) for x in text}
    sumi = 0
    for j,x in enumerate(mapp):
        sumi += len(mapp[x])
        if(sumi >= encoded_pos.item()):
            return j
    return len(mapp)

# Function for evaluation
def evaluate(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
            text = batch['text']
            
            outputs = model(input_ids, attention_mask=attention_mask)
            encoded_pos = torch.argmax(outputs.logits, dim=1)
            for i in range(len(text)):
                pos = find_position(text[i],encoded_pos[i],tokenizer)
                all_preds.append(pos)
            all_labels.extend(labels.cpu().numpy())
            
            del input_ids, attention_mask, labels, outputs
            torch.cuda.empty_cache()
    return all_preds,all_labels

In [33]:
num_epochs = 5
for epoch in range(num_epochs):
    train(new_model, optimizer, criterion2, train_loader, 1)
    all_preds,all_labels = evaluate(new_model, eval_loader)
    all_preds = torch.tensor(all_preds).to(torch.float32)
    all_labels = torch.tensor(all_labels).to(torch.float32)
    loss =torch.sqrt(criterion2(all_preds,all_labels))
    print(f"Epoch {epoch + 1}/{num_epochs}, Test Loss: {loss:.4f}")
    test_loss.append(loss)

Epoch 1/1, Loss: 185.2274


Evaluating: 100%|██████████| 64/64 [00:47<00:00,  1.35it/s]


Epoch 1/5, Test Loss: 75.4671


Epoch 1/1, Loss: 185.3130


Evaluating: 100%|██████████| 64/64 [00:47<00:00,  1.35it/s]


...

Evaluating: 100%|██████████| 64/64 [00:47<00:00,  1.34it/s]


Epoch 4/5, Test Loss: 75.4671


Epoch 1/1, Loss: 184.1484


Evaluating: 100%|██████████| 64/64 [00:47<00:00,  1.34it/s]

Epoch 5/5, Test Loss: 75.4671


In [42]:
for name, param in new_model.named_parameters():
    if 'classifier' not in name:  # If the parameter does not belong to the classifier layer
        param.requires_grad = False

# Verify that parameters are frozen as intended
for name, param in new_model.named_parameters():
    print(f'{name}: {param.requires_grad}')

roberta.embeddings.word_embeddings.weight: False
roberta.embeddings.position_embeddings.weight: False
roberta.embeddings.token_type_embeddings.weight: False
roberta.embeddings.LayerNorm.weight: False
roberta.embeddings.LayerNorm.bias: False
roberta.encoder.layer.0.attention.self.query.weight: False
roberta.encoder.layer.0.attention.self.query.bias: False
roberta.encoder.layer.0.attention.self.key.weight: False
roberta.encoder.layer.0.attention.self.key.bias: False
roberta.encoder.layer.0.attention.self.value.weight: False
roberta.encoder.layer.0.attention.self.value.bias: False
roberta.encoder.layer.0.attention.output.dense.weight: False
roberta.encoder.layer.0.attention.output.dense.bias: False
roberta.encoder.layer.0.attention.output.LayerNorm.weight: False
roberta.encoder.layer.0.attention.output.LayerNorm.bias: False
roberta.encoder.layer.0.intermediate.dense.weight: False
roberta.encoder.layer.0.intermediate.dense.bias: False
roberta.encoder.layer.0.output.dense.weight: False
robe

In [43]:
optimizer

AdamW (
Parameter Group 0
    betas: (0.9, 0.999)
    correct_bias: True
    eps: 1e-06
    lr: 0.001
    weight_decay: 0.0
)

In [44]:
num_epochs = 15
for epoch in range(num_epochs):
    train(new_model, optimizer, criterion2, train_loader, 1)
    all_preds,all_labels = evaluate(new_model, eval_loader)
    all_preds = torch.tensor(all_preds).to(torch.float32)
    all_labels = torch.tensor(all_labels).to(torch.float32)
    loss =torch.sqrt(criterion2(all_preds,all_labels))
    L1_loss =criterion3(all_preds,all_labels)
    print('l1 loss: ',L1_loss)
    print(f"Epoch {epoch + 1}/{num_epochs}, Test Loss: {loss:.4f}")
    test_loss.append(loss)

Epoch 1/1, Loss: 182.6813


Evaluating: 100%|██████████| 64/64 [00:48<00:00,  1.33it/s]


l1 loss:  tensor(49.6059)
Epoch 1/15, Test Loss: 75.4671


Epoch 1/1, Loss: 186.0201


Evaluating: 100%|██████████| 64/64 [00:47<00:00,  1.34it/s]


l1 loss:  tensor(49.6059)
Epoch 2/15, Test Loss: 75.4671


Epoch 1/1, Loss: 184.4204


Evaluating: 100%|██████████| 64/64 [00:47<00:00,  1.34it/s]


l1 loss:  tensor(49.6059)
Epoch 3/15, Test Loss: 75.4671


KeyboardInterrupt: 

In [45]:
# Save the trained model
output_dir = "/kaggle/working/"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("Model saved successfully.")

# Load the saved model
loaded_model = XLMRobertaForSequenceClassification.from_pretrained(output_dir)
loaded_tokenizer = XLMRobertaTokenizer.from_pretrained(output_dir)

Model saved successfully.
